In [12]:
!pip install --upgrade pip

In [21]:
!pip install ./openai-1.7.2

Processing ./openai-1.7.2
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      ERROR: Could not find a version that satisfies the requirement hatchling (from versions: none)
      ERROR: No matching distribution found for hatchling
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [13]:
!pip install ./pbr-6.0.0

Processing ./pbr-6.0.0
  Preparing metadata (setup.py) ... done
  Created wheel for pbr: filename=pbr-6.0.0-py2.py3-none-any.whl size=107509 sha256=0a9cc3a0235ec26fe2e6e372a5b64231f9221d9b2c44b592c9b74a4d00407bc7
  Stored in directory: /root/.cache/pip/wheels/1c/c5/2f/6c29070c96f86bfdaf0eebedfa26f14aae4bbb3196e75dcbaa
Successfully built pbr
  Attempting uninstall: pbr
    Found existing installation: pbr 6.0.0
    Uninstalling pbr-6.0.0:
      Successfully uninstalled pbr-6.0.0


In [14]:
!pip install ./msal-1.26.0

Processing ./msal-1.26.0
  Preparing metadata (setup.py) ... done
  Created wheel for msal: filename=msal-1.26.0-py2.py3-none-any.whl size=99026 sha256=2ee191e8ee048c731a76507f5bddfb1f64a2a7d6762dc1ce149b14414c588c12
  Stored in directory: /root/.cache/pip/wheels/5a/7b/43/10c5b10a51a72a42face8cf3fabcd9fe4f85eb233a7151dd65
Successfully built msal
  Attempting uninstall: msal
    Found existing installation: msal 1.26.0
    Uninstalling msal-1.26.0:
      Successfully uninstalled msal-1.26.0


In [15]:
#!pip install ./retry-0.9.2

Processing ./retry-0.9.2
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of retry to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement py<2.0.0,>=1.4.26 (from retry) (from versions: none)
ERROR: No matching distribution found for py<2.0.0,>=1.4.26


In [17]:
import os
import openai
import requests
import json
#from retry import retry
from msal import ClientApplication
import time

In [18]:
#@retry (tries=3, delay=2)
def getapikey():
    client_id = os.getenv("AZURE_CLIENT_ID")
    tenant_id = os.getenv("AZURE_TENANT_ID")
    endpoint = os.getenv("AZURE_ENDPOINT")
    scopes = [os.getenv("AZURE_APPLICATION_SCOPE")]
    username=os.getenv("SVC_ACCOUNT")
    password=os.getenv("SVC_PASSWORD")

    app = ClientApplication(
        client_id=client_id,
        authority="https://login.microsoftonline.com/" + tenant_id
    )

    acquire_tokens_result = app.acquire_token_by_username_password(username=username,
        password=password,
        scopes=scopes)

    if 'error' in acquire_tokens_result:
        print("Error: " + acquire_tokens_result['error'])
        print("Description: " + acquire_tokens_result['error_description'])
        return(2)
    else:
        header_token = {"Authorization": "Bearer {}".format(acquire_tokens_result['access_token'])}
        rt = requests.post(url=endpoint, headers=header_token, data=b'{"key":"openaikey2"}')
        return(rt.json())

In [19]:
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
# We are dynamically getting the key from AZURE.  Access is based on the service account/ad group combination
openai.api_key = getapikey()

NameError: name 'openai' is not defined